1. Import and Data Preparation

In [ ]:
import pandas as pd
import re

# Example DataFrames for deals and incidents
deals_data = {
    'deal_id': ["ELA-1268", "ALO-2289", "listfzhifdjnghnzri@@cnc@TD_XKDI", "ELA-8526", "edhbnskfbhrjd#20256@@FJNK"],
    'portfolio_id': ['AZE', 'AZE', 'BJU', 'BJU', 'CII'],
    'feature1': [10, 20, 10, 20, 30],
    'feature2': ['XEK', 'YCK', 'XEK', 'YCK', 'ZOP']
}
deals_df = pd.DataFrame(deals_data)

incidents_data = {
    'incident_id': [101, 102, 103],
    'group_or_portfolio': ['A', 'A', 'A'],
    'comment': [
        'Issue with deal ELA-1268 in portfolio AZE',
        'Issue on YCK: portfolio BJU is affected',
        'Entire CII has an incident on 12 MAR 2023'
    ]
}
incidents_df = pd.DataFrame(incidents_data)


: 

2. Comment Preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary nltk resources
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_comment(comment):
    # Tokenization without converting everything to lowercase
    tokens = word_tokenize(comment)
    # Remove punctuation and short tokens
    tokens = [word for word in tokens if word.isalnum() and len(word) > 1]
    # Remove stopwords except specified ones
    custom_stopwords = set(stopwords.words('english')) - {'all', 'entire'}
    tokens = [word for word in tokens if word.lower() not in custom_stopwords]
    return tokens

incidents_df['processed_comment'] = incidents_df['comment'].apply(preprocess_comment)
print(incidents_df['processed_comment'])


: 

3. Information Extraction from Comments
Extract relevant information based on identified tokens.

In [ ]:
def extract_info_from_comment(tokens, deals_df):
    deal_ids = set()
    portfolio_ids = set()
    features = {}
    
    for token in tokens:
        # Check if the token is a deal, portfolio, or group ID
        if token in deals_df['deal_id'].values:
            deal_ids.add(token)
        elif token in deals_df['portfolio_id'].values:
            portfolio_ids.add(token)
        elif token.isupper() and len(token) > 3:
            if token in deals_df['portfolio_id'].values:
                portfolio_ids.add(token)
        elif '=' in token:
            feature, value = token.split('=')
            if feature in deals_df.columns:
                features[feature] = value
    
    # Identify impacted deals
    affected_deals = set()
    
    if deal_ids:
        affected_deals.update(deals_df[deals_df['deal_id'].isin(deal_ids)].index)
    if portfolio_ids:
        affected_deals.update(deals_df[deals_df['portfolio_id'].isin(portfolio_ids)].index)
    for feature, value in features.items():
        if value.isdigit():
            value = int(value)
        affected_deals.update(deals_df[deals_df[feature] == value].index)
    
    return list(affected_deals)

# Example extraction
incident_example = incidents_df.iloc[1]
affected_deals = extract_info_from_comment(incident_example['processed_comment'], deals_df)
print(affected_deals)


: 

4. Association of Incidents with Deals and Flagging Impacted Deals

In [ ]:
def flag_deals(deals_df, incidents_df):
    deals_df['flag'] = 0
    
    for _, incident in incidents_df.iterrows():
        affected_deals = extract_info_from_comment(incident['processed_comment'], deals_df)
        deals_df.loc[affected_deals, 'flag'] = 1
    
    return deals_df

flagged_deals_df = flag_deals(deals_df, incidents_df)
print(flagged_deals_df)


: 